In [ ]:
import numpy as np

def matriz_transformacion(i, theta, parametros_dh):
    """
    Calcula la matriz de transformación para la articulación i utilizando parámetros DH.
    """
    a, alpha, d = parametros_dh[i-1][:3]
    if i == 2:  # Caso especial para la articulación 2 con un desfase de 90°
        theta = theta + 90

    theta_rad = np.radians(theta)
    alpha_rad = np.radians(alpha)

    return np.array([
        [np.cos(theta_rad), -np.sin(theta_rad)*np.cos(alpha_rad), np.sin(theta_rad)*np.sin(alpha_rad), a*np.cos(theta_rad)],
        [np.sin(theta_rad), np.cos(theta_rad)*np.cos(alpha_rad), -np.cos(theta_rad)*np.sin(alpha_rad), a*np.sin(theta_rad)],
        [0, np.sin(alpha_rad), np.cos(alpha_rad), d],
        [0, 0, 0, 1]
    ])

def obtener_eje_z(T):
    """Extrae el eje z de la matriz de transformación."""
    return T[:3, 2]

def obtener_posicion(T):
    """Extrae la posición de la matriz de transformación."""
    return T[:3, 3]

def calcular_jacobiano(theta1, theta2, theta3, theta4, parametros_dh):
    """
    Calcula la matriz Jacobiana para un robot de 4 grados de libertad (posición + un DOF rotacional).
    """
    T1 = matriz_transformacion(1, theta1, parametros_dh)
    T2 = matriz_transformacion(2, theta2, parametros_dh)
    T3 = matriz_transformacion(3, theta3, parametros_dh)
    T4 = matriz_transformacion(4, theta4, parametros_dh)

    T0_1 = T1
    T0_2 = T0_1 @ T2
    T0_3 = T0_2 @ T3
    T0_4 = T0_3 @ T4

    # Extraer posiciones
    p_e = obtener_posicion(T0_4)
    p0 = np.array([0, 0, 0])
    p1 = obtener_posicion(T0_1)
    p2 = obtener_posicion(T0_2)
    p3 = obtener_posicion(T0_3)

    # Extraer ejes z
    z0 = np.array([0, 0, 1])
    z1 = obtener_eje_z(T0_1)
    z2 = obtener_eje_z(T0_2)
    z3 = obtener_eje_z(T0_3)

    # Calcular Jacobiano de velocidad lineal
    Jv1 = np.cross(z0, p_e - p0)
    Jv2 = np.cross(z1, p_e - p1)
    Jv3 = np.cross(z2, p_e - p2)
    Jv4 = np.cross(z3, p_e - p3)

    # Seleccionar un eje de rotación útil (por ejemplo, alrededor del eje Z o X)
    Jw = np.vstack([z0, z1, z2, z3])[:, 2]  # Mantener la rotación alrededor del eje Z

    # Ensamblar la matriz Jacobiana completa (4×4)
    J = np.vstack([
        np.column_stack([Jv1, Jv2, Jv3, Jv4]),  # Velocidades lineales (3×4)
        Jw  # 1 DOF rotacional (1×4)
    ])

    return J  # Ahora es una matriz 4×4
    J /= np.linalg.norm(J, axis=0)  # Normalizar cada columna

def analizar_jacobiano(theta1, theta2, theta3, theta4):
    """
    Analiza la matriz Jacobiana e imprime información relevante.
    """
    # Parámetros DH [a, alpha, d] para cada articulación
    parametros_dh = [
        [0, -90, 35],   # Articulación 1
        [30, 0, 0],     # Articulación 2
        [0, 90, -30],   # Articulación 3
        [0, -90, 1]     # Articulación 4
    ]

    # Calcular el Jacobiano
    J = calcular_jacobiano(theta1, theta2, theta3, theta4, parametros_dh)

    print("Matriz Jacobiana:")
    print(np.round(J, 3))

    # Calcular e imprimir la medida de manipulabilidad
    w = np.sqrt(np.linalg.det(J @ J.T))
    print(f"\nMedida de manipulabilidad: {w:.3f}")

    # Calcular el número de condición
    try:
        cond = np.linalg.cond(J)
        print(f"Número de condición: {cond:.3f}")
    except np.linalg.LinAlgError:
        print("Número de condición: Configuración singular")

    # Verificar singularidades
    rango = np.linalg.matrix_rank(J)
    print(f"Rango de la matriz: {rango}")
    if rango < 4:
        print("¡El robot está en una configuración singular!")

if __name__ == "__main__":
    # Definir los parámetros DH dentro de __main__
    parametros_dh = [
        [0, -90, 35],   # Articulación 1
        [30, 0, 0],     # Articulación 2
        [0, 90, -30],   # Articulación 3
        [0, -90, 1]     # Articulación 4
    ]

    # Ángulos de prueba existentes
    angulos_prueba = [30, 45, -30, 60]  # Ejemplo de ángulos originales
    print("Analizando configuración original:")
    print("=" * 50)
    analizar_jacobiano(*angulos_prueba)

    # ✅ Paso 3: Optimizar los ángulos de las articulaciones para un mejor número de condición
    from scipy.optimize import minimize

    def numero_de_condicion(angulos, parametros_dh):
        """Calcula el número de condición del Jacobiano para minimizarlo."""
        J = calcular_jacobiano(*angulos, parametros_dh)
        return np.linalg.cond(J)  # Minimizar el número de condición

    # Suposición inicial (misma que los ángulos de prueba)
    angulos_iniciales = angulos_prueba

    # Límites para los ángulos (ajustar según límites articulares)
    limites = [(0, 180), (-180, 180), (-90, 90), (-180, 180)]

    print("\nOptimizando los ángulos de las articulaciones para mejorar la condición del Jacobiano...")
    resultado = minimize(numero_de_condicion, angulos_iniciales, bounds=limites, args=(parametros_dh,))

    # Extraer los ángulos optimizados
    angulos_optimizados = resultado.x
    print("\nÁngulos optimizados:", angulos_optimizados)

    # Analizar la configuración optimizada
    print("\nAnalizando configuración optimizada:")
    print("=" * 50)
    analizar_jacobiano(*angulos_optimizados)

    # Probar configuraciones adicionales
    configuraciones_prueba_v2 = [
        [45, 90, -60, 30],   # Ángulos más extremos
        [60, 75, -45, 15],   # Mayor diferencia entre articulaciones
        [30, 120, -60, 45]   # Ángulo theta2 muy grande
    ]

    for angulos in configuraciones_prueba_v2:
        print(f"\nProbando ángulos: {angulos}")
        analizar_jacobiano(*angulos)